# Human numbers

In [1]:

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.text import *

from fastai import *

In [3]:
bs=64

## Data

In [4]:
untar_data??

In [4]:
path = untar_data(URLs.HUMAN_NUMBERS)
path.ls()

[PosixPath('/home/jupyter/.fastai/data/human_numbers/valid.txt'),
 PosixPath('/home/jupyter/.fastai/data/human_numbers/train.txt'),
 PosixPath('/home/jupyter/.fastai/data/human_numbers/models')]

In [17]:
valid = 'valid.txt'
! head {path}/{valid}

eight thousand one 
eight thousand two 
eight thousand three 
eight thousand four 
eight thousand five 
eight thousand six 
eight thousand seven 
eight thousand eight 
eight thousand nine 
eight thousand ten 


In [18]:
! head {path}/'train.txt'

one 
two 
three 
four 
five 
six 
seven 
eight 
nine 
ten 


In [24]:
'  eight'.strip()

'eight'

In [5]:
def readnums(d): return [', '.join(o.strip() for o in open(path/d).readlines())]

In [6]:
train_txt = readnums('train.txt')

In [8]:
len(train_txt)

1

In [9]:
train_txt[0][:80]

'one, two, three, four, five, six, seven, eight, nine, ten, eleven, twelve, thirt'

In [10]:
train_txt[0][:5]

'one, '

In [7]:
valid_txt = readnums('valid.txt'); valid_txt[0][-80:]

' nine thousand nine hundred ninety eight, nine thousand nine hundred ninety nine'

In [12]:
len(valid_txt[0].split(","))

1999

In [34]:
doc(TextList)

In [35]:
path.parts

('/', 'home', 'jupyter', '.fastai', 'data', 'human_numbers')

In [37]:
type(train_txt)

list

In [40]:
array(train_txt, dtype=object)[0][:5]

'one, '

In [8]:
train = TextList(train_txt, path=path)
valid = TextList(valid_txt, path=path)

src = ItemLists(path=path, train=train, valid=valid).label_for_lm()
data = src.databunch(bs=bs)

In [29]:
train[0].text[:80]

'xxbos one , two , three , four , five , six , seven , eight , nine , ten , eleve'

In [30]:
data.batch_size

64

In [52]:
data.train_ds.y

LMLabel (1 items)
[Category 0]...
Path: /home/jupyter/.fastai/data/human_numbers

In [63]:
data.train_ds[0][0].data

array([ 2, 11,  8, 12, ..., 19,  9, 27, 19])

In [64]:
len(data.train_ds[0][0].data)

50079

In [65]:
data.valid_ds[0][0].data

array([ 2, 18, 10, 11, ..., 19,  9, 27, 19])

### There are 13,017 tokens in the validation set (including the commas). Tokens are each item in the doc separated by a space

In [31]:
len(data.valid_ds[0][0].data)

13017

In [9]:
data.bptt, len(data.valid_dl)

(70, 3)

In [68]:
data.valid_dl

DeviceDataLoader(dl=<fastai.text.data.LanguageModelLoader object at 0x7fba58b941d0>, device=device(type='cuda'), tfms=[], collate_fn=<function data_collate at 0x7fba622ec950>)

In [70]:
len(data.train_dl)

12

### Our dataset of 13k tokens is broken up into 64 segments (since batch size of 64) and then we divide into bptt groups of about 70 (randomized a bit on the size) to give us 3 actual batches from the validation set

In [71]:
13017/70/bs

2.905580357142857

#### 13017 tokens in validation set divided by bptt divided by bs = 3 and we have 3 iters of the validation loader

In [33]:
it = iter(data.valid_dl)
x1,y1 = next(it)
x2,y2 = next(it)
x3,y3 = next(it)
it.close()

In [85]:
it2 = iter(data.valid_dl)

In [86]:
f1,g1 = next(it2)

#### f1 is our independent variables and g1 is our dependent variables. so the g1's are just shifted one over from the f1's since predicting the next word

In [87]:
f1, g1

(tensor([[ 2, 18, 10,  ..., 33,  8, 18],
         [ 8, 18, 10,  ...,  8, 18, 10],
         [10, 26, 17,  ..., 10, 27, 19],
         ...,
         [10, 19,  9,  ..., 19, 10, 19],
         [ 9, 21, 17,  ..., 19, 10, 19],
         [ 8, 19, 10,  ...,  9, 25, 15]], device='cuda:0'),
 tensor([[18, 10, 11,  ...,  8, 18, 10],
         [18, 10, 22,  ..., 18, 10, 23],
         [26, 17,  8,  ..., 27, 19,  8],
         ...,
         [19,  9, 16,  ..., 10, 19,  9],
         [21, 17,  8,  ..., 10, 19,  9],
         [19, 10, 19,  ..., 25, 15,  8]], device='cuda:0'))

#### get an error if iterate 4 times since we only have 3 iterations

In [88]:
f2,g2 = next(it2)
f3,g3 = next(it2)
f4,g4 = next(it2)

StopIteration: 

In [90]:
torch.numel??
# shows us number of elements in tensor. so for 4x4 matrix, 16 numel

### We have 12,928 tokens in the 3 batches rather than the original 13,017 tokens since the excess didn't fit into the last batch

In [34]:
x1.numel()+x2.numel()+x3.numel()

12928

In [108]:
x1.numel()

6080

In [109]:
95 * 64

6080

In [35]:
y1.numel()

6080

### 64 is our batch size and 95 represents our bptt here. The data loader randomizes it so for each batch it will be around 70 rather than exactly 70

In [106]:
x1.shape,y1.shape

(torch.Size([64, 95]), torch.Size([64, 95]))

In [107]:
x2.shape,y2.shape

(torch.Size([64, 77]), torch.Size([64, 77]))

In [44]:
x1.shape

torch.Size([64, 95])

In [51]:
x1

tensor([[ 2, 18, 10,  ..., 10, 20, 13],
        [ 8, 18, 10,  ..., 14,  8, 18],
        [10, 26, 17,  ..., 18, 10, 11],
        ...,
        [10, 19,  9,  ...,  9, 20, 11],
        [ 9, 21, 17,  ...,  8, 19, 10],
        [ 8, 19, 10,  ..., 10, 19,  9]], device='cuda:0')

In [50]:
# gives all rows of column 1

x1[:, 1]

tensor([18, 18, 26, 11, 12, 10, 12, 13, 10, 18, 14, 10, 27, 15, 10, 26, 10, 10,
        25,  8, 10, 24, 18, 10, 23, 18, 18, 22, 18, 18, 21, 18,  9, 10, 14, 11,
        23, 19, 19, 11, 10,  9, 12, 10, 27, 13, 10, 26,  8, 10, 25,  9, 10, 24,
        19, 10, 23, 19, 34, 22, 19, 19, 21, 19], device='cuda:0')

In [45]:
# gives all columns of row 0

x1[0,:]

tensor([ 2, 18, 10, 11,  8, 18, 10, 12,  8, 18, 10, 13,  8, 18, 10, 14,  8, 18,
        10, 15,  8, 18, 10, 16,  8, 18, 10, 17,  8, 18, 10, 18,  8, 18, 10, 19,
         8, 18, 10, 28,  8, 18, 10, 29,  8, 18, 10, 30,  8, 18, 10, 31,  8, 18,
        10, 32,  8, 18, 10, 33,  8, 18, 10, 34,  8, 18, 10, 35,  8, 18, 10, 36,
         8, 18, 10, 37,  8, 18, 10, 20,  8, 18, 10, 20, 11,  8, 18, 10, 20, 12,
         8, 18, 10, 20, 13], device='cuda:0')

In [115]:
# this is what we want to pull our tokens in order. we see that y1[0] is offset by 1

x1[0]

tensor([ 2, 18, 10, 11,  8, 18, 10, 12,  8, 18, 10, 13,  8, 18, 10, 14,  8, 18,
        10, 15,  8, 18, 10, 16,  8, 18, 10, 17,  8, 18, 10, 18,  8, 18, 10, 19,
         8, 18, 10, 28,  8, 18, 10, 29,  8, 18, 10, 30,  8, 18, 10, 31,  8, 18,
        10, 32,  8, 18, 10, 33,  8, 18, 10, 34,  8, 18, 10, 35,  8, 18, 10, 36,
         8, 18, 10, 37,  8, 18, 10, 20,  8, 18, 10, 20, 11,  8, 18, 10, 20, 12,
         8, 18, 10, 20, 13], device='cuda:0')

In [57]:
y1[0]

tensor([18, 10, 11,  8, 18, 10, 12,  8, 18, 10, 13,  8, 18, 10, 14,  8, 18, 10,
        15,  8, 18, 10, 16,  8, 18, 10, 17,  8, 18, 10, 18,  8, 18, 10, 19,  8,
        18, 10, 28,  8, 18, 10, 29,  8, 18, 10, 30,  8, 18, 10, 31,  8, 18, 10,
        32,  8, 18, 10, 33,  8, 18, 10, 34,  8, 18, 10, 35,  8, 18, 10, 36,  8,
        18, 10, 37,  8, 18, 10, 20,  8, 18, 10, 20, 11,  8, 18, 10, 20, 12,  8,
        18, 10, 20, 13,  8], device='cuda:0')

In [36]:
# gives all rows of column 0. not what we want

x1[:,0]

tensor([ 2,  8, 10, 10, 23, 18, 10, 22, 18,  8, 21, 18,  9, 20, 18,  9, 18, 18,
         9, 15, 18,  9,  8, 18,  9,  8,  8,  9,  8,  9,  9,  8, 19, 19, 26, 10,
         9,  8,  8, 22, 19, 13, 21, 19,  9, 20, 19,  9, 31, 19,  9, 16, 19,  9,
         8, 19,  9,  8,  9,  9,  8, 10,  9,  8], device='cuda:0')

In [58]:
y1[:, 0]

tensor([18, 18, 26, 11, 12, 10, 12, 13, 10, 18, 14, 10, 27, 15, 10, 26, 10, 10,
        25,  8, 10, 24, 18, 10, 23, 18, 18, 22, 18, 18, 21, 18,  9, 10, 14, 11,
        23, 19, 19, 11, 10,  9, 12, 10, 27, 13, 10, 26,  8, 10, 25,  9, 10, 24,
        19, 10, 23, 19, 34, 22, 19, 19, 21, 19], device='cuda:0')

In [16]:
v = data.valid_ds.vocab

### The below shows that our minibatches match up. We end x1[0] and then x2[0] picks up from there and then x3[0]. Then x1[1] picks up around where x3[0] ended.

In [52]:
v.textify(x1[0])

'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three'

In [53]:
v.textify(y1[0])

'eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three ,'

In [54]:
v.textify(x2[0])

', eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two , eight thousand thirty three , eight thousand thirty four , eight thousand thirty five , eight thousand thirty six ,'

In [55]:
v.textify(x3[0])

'eight thousand thirty seven , eight thousand thirty eight , eight thousand thirty nine , eight thousand forty , eight thousand forty one , eight thousand forty two , eight thousand forty three , eight thousand forty four , eight thousand forty'

In [124]:
v.textify(x1[1])

', eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight thousand sixty , eight thousand sixty one , eight thousand sixty two , eight thousand sixty three , eight thousand sixty four , eight'

In [125]:
v.textify(x2[1])

'thousand sixty five , eight thousand sixty six , eight thousand sixty seven , eight thousand sixty eight , eight thousand sixty nine , eight thousand seventy , eight thousand seventy one , eight thousand seventy two , eight thousand seventy three , eight thousand seventy four , eight thousand seventy five , eight thousand seventy six , eight thousand seventy seven , eight thousand seventy eight , eight thousand seventy nine , eight thousand eighty ,'

In [59]:
v.textify(x3[1])

'seventy eight , eight thousand seventy nine , eight thousand eighty , eight thousand eighty one , eight thousand eighty two , eight thousand eighty three , eight thousand eighty four , eight thousand eighty five , eight thousand eighty six ,'

In [128]:
v.textify(x3[-1])

', nine thousand nine hundred ninety two , nine thousand nine hundred ninety three , nine thousand nine hundred ninety four , nine thousand nine hundred ninety five , nine'

In [126]:
data.show_batch(ds_type=DatasetType.Valid)

idx,text
0,"xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen"
1,", eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight"
2,"thousand eighty seven , eight thousand eighty eight , eight thousand eighty nine , eight thousand ninety , eight thousand ninety one , eight thousand ninety two , eight thousand ninety three , eight thousand ninety four , eight thousand ninety five , eight thousand ninety six , eight thousand ninety seven , eight thousand ninety eight , eight thousand ninety nine , eight"
3,"thousand one hundred twenty three , eight thousand one hundred twenty four , eight thousand one hundred twenty five , eight thousand one hundred twenty six , eight thousand one hundred twenty seven , eight thousand one hundred twenty eight , eight thousand one hundred twenty nine , eight thousand one hundred thirty , eight thousand one hundred thirty one , eight thousand one"
4,"fifty two , eight thousand one hundred fifty three , eight thousand one hundred fifty four , eight thousand one hundred fifty five , eight thousand one hundred fifty six , eight thousand one hundred fifty seven , eight thousand one hundred fifty eight , eight thousand one hundred fifty nine , eight thousand one hundred sixty , eight thousand one hundred sixty one"


## Single fully connected model

In [10]:
data = src.databunch(bs=bs, bptt=3, max_len=0, p_bptt=1.)

In [14]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 3]), torch.Size([64, 3]))

In [64]:
x

tensor([[ 2, 11,  8],
        [ 9, 28,  8],
        [15,  8, 13],
        [15,  9, 22],
        [17,  9, 18],
        [ 8, 18,  9],
        [10, 22, 11],
        [11,  9, 25],
        [12,  9, 27],
        [ 9, 17,  8],
        [15,  9, 20],
        [ 9, 22, 11],
        [ 9, 23, 17],
        [25, 13,  8],
        [26, 19,  8],
        [ 9, 21, 17],
        [23, 13,  8],
        [24, 19,  8],
        [15,  8, 12],
        [10, 16,  9],
        [20,  8, 12],
        [16,  8, 12],
        [ 8, 12, 10],
        [13, 10, 27],
        [10, 12,  9],
        [ 8, 13, 10],
        [ 8, 13, 10],
        [13, 10, 15],
        [10, 16,  9],
        [10, 17,  9],
        [19,  9, 33],
        [10, 22, 15],
        [10, 11,  9],
        [12,  9, 27],
        [ 8, 14, 10],
        [10, 15,  9],
        [16,  9, 22],
        [17,  9, 24],
        [ 9, 25, 16],
        [27, 12,  8],
        [11,  9, 22],
        [ 9, 23, 16],
        [25, 12,  8],
        [26, 18,  8],
        [ 9, 15,  8],
        [2

In [65]:
y

tensor([[11,  8, 12],
        [28,  8, 12],
        [ 8, 13,  9],
        [ 9, 22, 13],
        [ 9, 18,  8],
        [18,  9, 25],
        [22, 11,  8],
        [ 9, 25, 14],
        [ 9, 27,  8],
        [17,  8, 11],
        [ 9, 20, 15],
        [22, 11,  8],
        [23, 17,  8],
        [13,  8, 11],
        [19,  8, 11],
        [21, 17,  8],
        [13,  8, 12],
        [19,  8, 12],
        [ 8, 12, 10],
        [16,  9, 12],
        [ 8, 12, 10],
        [ 8, 12, 10],
        [12, 10, 19],
        [10, 27, 17],
        [12,  9, 34],
        [13, 10, 13],
        [13, 10, 14],
        [10, 15,  9],
        [16,  9, 26],
        [17,  9, 27],
        [ 9, 33,  8],
        [22, 15,  8],
        [11,  9, 25],
        [ 9, 27, 13],
        [14, 10, 14],
        [15,  9, 20],
        [ 9, 22, 14],
        [ 9, 24,  8],
        [25, 16,  8],
        [12,  8, 14],
        [ 9, 22,  8],
        [23, 16,  8],
        [12,  8, 15],
        [18,  8, 15],
        [15,  8, 15],
        [1

In [19]:
nv = len(v.itos); nv

38

In [69]:
v.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 ',',
 'hundred',
 'thousand',
 'one',
 'two',
 'three',
 'four',
 'five',
 'six',
 'seven',
 'eight',
 'nine',
 'twenty',
 'thirty',
 'forty',
 'fifty',
 'sixty',
 'seventy',
 'eighty',
 'ninety',
 'ten',
 'eleven',
 'twelve',
 'thirteen',
 'fourteen',
 'fifteen',
 'sixteen',
 'seventeen',
 'eighteen',
 'nineteen']

In [11]:
nh=64

In [12]:
def loss4(input,target): return F.cross_entropy(input, target[:,-1])
def acc4 (input,target): return accuracy(input, target[:,-1])

In [72]:
accuracy??

In [73]:
F.cross_entropy??

### We're now doing a batch size of 64 with bptt of 3

In [15]:
x.shape

torch.Size([64, 3])

In [130]:
class Model0(nn.Module):
    def __init__(self):
        super().__init__()
#         input to hidden
        self.i_h = nn.Embedding(nv,nh)  # green arrow
#         hidden to hidden
        self.h_h = nn.Linear(nh,nh)     # brown arrow
#     hidden to output
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
#         print("x.shape!!", x.shape)
        h = self.bn(F.relu(self.i_h(x[:,0])))
#         print("h!!", h)
#         at the end of the batch, could have fewer than 3 words left so need this condition
        if x.shape[1]>1:
#         for all of the first words for the 64 batch, add an input to the hidden layer using that first word 
            h = h + self.i_h(x[:,1])
            h = self.bn(F.relu(self.h_h(h)))
        if x.shape[1]>2:
#         for all of the second words for the 64 batch, add an input to the hidden layer using that second word 
            h = h + self.i_h(x[:,2])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [131]:
learn = Learner(data, Model0(), loss_func=loss4, metrics=acc4)

In [132]:
learn.fit_one_cycle(2, 1e-4)

epoch,train_loss,valid_loss,acc4
1,3.401159,3.492699,0.094210
2,3.098823,3.353392,0.257123


## Same thing with a loop

In [153]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
#         we change the initialization of h here to all 0s which is improving performance somehow?
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
#     prior h initialization:
#         h = self.bn(F.relu(self.i_h(x[:,0])))
#         print("h!!", h)

# same thing as in prior model but have for loop
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [154]:
learn = Learner(data, Model1(), loss_func=loss4, metrics=acc4)

In [155]:
learn.fit_one_cycle(2, 1e-4)

epoch,train_loss,valid_loss,acc4
1,3.274400,3.316405,0.268612
2,2.955998,3.130739,0.381893


## Multi fully connected model

In [37]:
data = src.databunch(bs=bs, bptt=20)

In [35]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 45]), torch.Size([64, 45]))

In [32]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 18]), torch.Size([64, 18]))

In [28]:
accuracy

<function fastai.metrics.accuracy(input:torch.Tensor, targs:torch.LongTensor) -> <function NewType.<locals>.new_type at 0x7f1d74528c80>>

In [36]:
data.one_batch??

### Why 45?? If have 20 bptt??  It is never exactly 20, randomizes a bit and looks like first batch is large.

In [38]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        res = []
        print("x.shape!!", x.shape)
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.h_o(self.bn(h)))
        return torch.stack(res, dim=1)

### Now using default accuracy and loss function instead of just checking against one output at the end. Check for every word

In [39]:
learn = Learner(data, Model2(), metrics=accuracy)

In [41]:
learn.fit_one_cycle(2, 1e-4, pct_start=0.1)

epoch,train_loss,valid_loss,accuracy
1,3.566423,3.594486,0.110885
2,3.508735,3.569892,0.117124


x.shape!! torch.Size([64, 17])
x.shape!! torch.Size([64, 26])
x.shape!! torch.Size([64, 22])
x.shape!! torch.Size([64, 14])
x.shape!! torch.Size([64, 24])
x.shape!! torch.Size([64, 18])
x.shape!! torch.Size([64, 26])
x.shape!! torch.Size([64, 25])
x.shape!! torch.Size([64, 18])
x.shape!! torch.Size([64, 10])
x.shape!! torch.Size([64, 22])
x.shape!! torch.Size([64, 19])
x.shape!! torch.Size([64, 19])
x.shape!! torch.Size([64, 18])
x.shape!! torch.Size([64, 12])
x.shape!! torch.Size([64, 17])
x.shape!! torch.Size([64, 23])
x.shape!! torch.Size([64, 18])
x.shape!! torch.Size([64, 19])
x.shape!! torch.Size([64, 21])
x.shape!! torch.Size([64, 11])
x.shape!! torch.Size([64, 23])
x.shape!! torch.Size([64, 15])
x.shape!! torch.Size([64, 10])
x.shape!! torch.Size([64, 23])
x.shape!! torch.Size([64, 12])
x.shape!! torch.Size([64, 26])
x.shape!! torch.Size([64, 22])
x.shape!! torch.Size([64, 17])
x.shape!! torch.Size([64, 17])
x.shape!! torch.Size([64, 19])
x.shape!! torch.Size([64, 13])
x.shape!

## Maintain state

### We move the initialization of the hidden layer to the constructor so we don't have to start from scratch on each batch. Since the batches are in order this works

In [60]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(x.shape[0], nh).cuda()
        
    def forward(self, x):
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        res = self.h_o(res)
        return res

In [61]:
learn = Learner(data, Model3(), metrics=accuracy)

In [62]:
learn.fit_one_cycle(1, 3e-3)

epoch,train_loss,valid_loss,accuracy
1,2.658973,2.319619,0.465813


In [22]:
sys.path

['',
 '/opt/anaconda3/lib/python36.zip',
 '/opt/anaconda3/lib/python3.6',
 '/opt/anaconda3/lib/python3.6/lib-dynload',
 '/home/jupyter/.local/lib/python3.6/site-packages',
 '/opt/anaconda3/lib/python3.6/site-packages',
 '/home/jupyter/.local/lib/python3.6/site-packages/IPython/extensions',
 '/home/jupyter/.ipython']

In [23]:
BatchNorm1dFlat

fastai.layers.BatchNorm1dFlat

In [27]:
nn.RNN??

## nn.RNN

In [39]:
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
#         nh for input_size and nh for hidden_size
        self.rnn = nn.RNN(nh,nh, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(1, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [40]:
learn = Learner(data, Model4(), metrics=accuracy)

In [41]:
learn.fit_one_cycle(2, 3e-3)

epoch,train_loss,valid_loss,accuracy
1,1.453558,2.060870,0.322610
2,1.430948,2.241736,0.318781


In [31]:
doc(nn.GRU)

## 2-layer GRU

In [28]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.GRU(nh,nh,2, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(2, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [29]:
learn = Learner(data, Model5(), metrics=accuracy)

In [30]:
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,1.464701,1.796802,0.482077
2,0.982733,2.206717,0.500000
3,0.791821,3.872705,0.454427
4,0.740694,3.659478,0.414062
5,0.756192,7.048537,0.413603
6,0.816509,2.766655,0.440411
7,0.866556,3.505463,0.421339
8,0.936464,5.850315,0.414752
9,0.956766,3.719980,0.464078
10,0.955129,3.378005,0.457108


## fin